In [10]:
#IMPORTAR BIBLIOTECAS UTILIZADAS

import numpy as np
import pandas as pd
import random
from collections import Counter
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

In [11]:
"""
    Classe para processar o dataset
    Obs: o dataset tem que ser para problemas de classificacao e nele tem que conter uma coluna Class
"""
import os
import sys
sys.path.insert(0, os.path.abspath('../Util'))
from dados import ProcessarDados

In [12]:
#IMPORTANDO DATASET

dataset = ProcessarDados('../dataset/norm_bin_10_FEATURES_M17_CM6b_TH199.csv')

In [13]:
# função que treina e testa o modelo armazenando as métricas
# retorna um dicionário cotendo os valores das métricas de cada rodada

def treinamento_teste(epocas = 10, k_folds = 5, exibir_matriz_confusao=False, exibir_metricas=False):
    
    #array para armazenar as das métricas de cada rodada
    resultados_accuracy = []
    resultados_precision = []
    resultados_recall = []
    resultados_specificity = [] # taxa de verdadeiros negativos ou especificidade
    resultados_f2 = []
    resultados_parametros = []
    
    #dicionário das métricas
    resultados_gerais = {}

    for i in range(epocas):
        #SEPARAR DADOS DE TREINO E TESTE MANTENDO A MESMA PROPORÇÃO  (80% E 20%)
        seed = i
        X_treino, X_teste, y_treino, y_teste = dataset.holdout(0.2, seed)
        
        #ALTERANDO DIMENSÃO DE Y PARA D=1

        y_treino = y_treino.reshape(480,)
        y_teste = y_teste.reshape(120,)
        
        # realizando o grid search para encontrar o melhor alpha, 
        # considerando a acurácia (taxa de acerto)
        # aqui o método GridSearchCV é configurado para subdividir os dados de treino em k_folds
        rl = LogisticRegression(random_state = seed)
        grid_rl = GridSearchCV(rl, param_grid, cv=k_folds, scoring='accuracy', verbose=0)
        grid_rl.fit(X_treino, y_treino)

        # Treinando do modelo com os melhores parametros encontrados
        alpha_best = grid_rl.best_estimator_.tol

        rl = LogisticRegression(random_state = seed, tol = alpha_best)
        rl.fit(X_treino, y_treino)

        #testando o modelo
        y_pred = rl.predict(X_teste)
        
        cm  = confusion_matrix(y_teste, y_pred)
        if exibir_matriz_confusao:
            print(cm)


        # calculado as metricas
        accuracy = metrics.accuracy_score(y_teste, y_pred)
        precision = metrics.precision_score(y_teste, y_pred)
        recall = metrics.recall_score(y_teste, y_pred)
        # f2-score
        # Fbeta = ((1 + beta^2) * Precision * Recall) / (beta^2 * Precision + Recall)
        beta = 0.5
        f2_score = (1 + beta**2) * (precision * recall) / (beta**2 * precision + recall)
        tn, fp, fn, tp = cm.ravel()
        specificity = tn / (tn+fp)
        

        # armazenando as métricas
        resultados_accuracy.append(accuracy)
        resultados_precision.append(precision)
        resultados_recall.append(recall)
        resultados_specificity.append(specificity)
        resultados_f2.append(f2_score)

        best_parametros = {'alpha' : alpha_best};
        resultados_parametros.append(best_parametros)
        
        if exibir_metricas:
            print("Rodada: #",i)
            print(best_parametros)
            print("Accuracy:",accuracy)
            print("Precision:",precision)
            print("Recall:",recall)
            print("Specificity:",specificity)
            print("f2-Score:",f2_score)
            print("\n")

            
    resultados_gerais['accuracy'] = resultados_accuracy
    resultados_gerais['precision'] = resultados_precision
    resultados_gerais['recall'] = resultados_recall
    resultados_gerais['specificity'] = resultados_specificity
    resultados_gerais['f2'] = resultados_f2
    resultados_gerais['params'] = resultados_parametros
    
    return resultados_gerais

In [14]:
def tabelaMetricas(nome_modelo, dict_metricas, rodadas=False, salvarResultados=True):
        
    print ("============================================== "+nome_modelo+" =================================================")
    print ("=================================== TABELA DE MÉTRICAS DO MODELO ===================================")
    
    if(rodadas==False):
        print ("\t Accuracy \t|\t Precision \t|\t Recall \t|\tSpecificity \t|\t fb-Score")
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['accuracy'], axis=0), np.std(dict_metricas['accuracy'], axis=0)),end='       ')
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['precision'], axis=0), np.std(dict_metricas['precision'], axis=0)),end='    ')
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['recall'], axis=0), np.std(dict_metricas['recall'], axis=0)),end='       ')
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['specificity'], axis=0), np.std(dict_metricas['specificity'], axis=0)),end='       ')
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['f2'], axis=0), np.std(dict_metricas['f2'], axis=0)))
        print ("====================================================================================================")
        
    if(salvarResultados):
        # save to npy file
        np.save('../resultados/resultados_'+nome_modelo+'.npy', dict_metricas)

In [15]:
#DEFININDO OS PARÂMETROS


alpha_range = np.logspace(-3,0,25)
param_grid = {'tol': alpha_range}

epocas = 50
k_folds = 5
exibir_matriz_confusao = False
exibir_metricas = True

# TREINANDO O MODELO E OBTENDO AS MÉTRICAS

dict_metricas = treinamento_teste(epocas, k_folds, exibir_matriz_confusao, exibir_metricas)
tabelaMetricas('RL',dict_metricas)

Rodada: # 0
{'alpha': 0.1333521432163324}
Accuracy: 0.9333333333333333
Precision: 0.9468085106382979
Recall: 0.967391304347826
Specificity: 0.8214285714285714
f2-Score: 0.9508547008547008


Rodada: # 1
{'alpha': 0.5623413251903491}
Accuracy: 0.8666666666666667
Precision: 0.865979381443299
Recall: 0.9655172413793104
Specificity: 0.6060606060606061
f2-Score: 0.8842105263157896


Rodada: # 2
{'alpha': 0.001}
Accuracy: 0.8833333333333333
Precision: 0.9285714285714286
Recall: 0.9285714285714286
Specificity: 0.6818181818181818
f2-Score: 0.9285714285714286


Rodada: # 3
{'alpha': 0.4216965034285822}
Accuracy: 0.8916666666666667
Precision: 0.8947368421052632
Recall: 0.9659090909090909
Specificity: 0.6875
f2-Score: 0.9081196581196581


Rodada: # 4
{'alpha': 0.31622776601683794}
Accuracy: 0.9333333333333333
Precision: 0.9375
Recall: 0.9782608695652174
Specificity: 0.7857142857142857
f2-Score: 0.9453781512605044


Rodada: # 5
{'alpha': 0.001}
Accuracy: 0.8666666666666667
Precision: 0.86
Recall: 0

Rodada: # 47
{'alpha': 0.001}
Accuracy: 0.9
Precision: 0.92
Recall: 0.9583333333333334
Specificity: 0.6666666666666666
f2-Score: 0.9274193548387095


Rodada: # 48
{'alpha': 0.001}
Accuracy: 0.9
Precision: 0.8888888888888888
Recall: 0.9887640449438202
Specificity: 0.6451612903225806
f2-Score: 0.9072164948453609


Rodada: # 49
{'alpha': 0.23713737056616552}
Accuracy: 0.8833333333333333
Precision: 0.8829787234042553
Recall: 0.9651162790697675
Specificity: 0.6764705882352942
f2-Score: 0.8982683982683982


============================================== RL =================================================
=================================== TABELA DE MÉTRICAS DO MODELO ===================================
	 Accuracy 	|	 Precision 	|	 Recall 	|	Specificity 	|	 fb-Score
      0.90 +- 0.02             0.91 +- 0.03          0.95 +- 0.02             0.72 +- 0.09             0.92 +- 0.03
